# RPM Assignment

In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

df = yf.download("BTC-USD", start="2022-01-01", end="2025-01-01")
df.dropna(inplace=True)
df.columns = df.columns.get_level_values(0)
df.head()


In [ ]:

n = 14
df['roc'] = df['Close'].pct_change(n) * 100


In [ ]:

df['nvi'] = np.nan
df.iloc[0, df.columns.get_loc('nvi')] = 1000

for i in range(1, len(df)):
    if df['Volume'].iloc[i] < df['Volume'].iloc[i-1]:
        df.iloc[i, df.columns.get_loc('nvi')] = (
            df['nvi'].iloc[i-1] *
            (1 + (df['Close'].iloc[i] - df['Close'].iloc[i-1]) / df['Close'].iloc[i-1])
        )
    else:
        df.iloc[i, df.columns.get_loc('nvi')] = df['nvi'].iloc[i-1]


In [ ]:

def ps(h, l, a=0.02, m=0.2):
    p = h.copy()
    b = True
    af = a
    ep = l.iloc[0]
    p.iloc[0] = l.iloc[0]

    for i in range(1, len(h)):
        pr = p.iloc[i-1]
        if b:
            p.iloc[i] = pr + af * (ep - pr)
            if l.iloc[i] < p.iloc[i]:
                b = False
                p.iloc[i] = ep
                af = a
                ep = h.iloc[i]
        else:
            p.iloc[i] = pr + af * (ep - pr)
            if h.iloc[i] > p.iloc[i]:
                b = True
                p.iloc[i] = ep
                af = a
                ep = l.iloc[i]

        if b and h.iloc[i] > ep:
            ep = h.iloc[i]
            af = min(af + a, m)
        elif not b and l.iloc[i] < ep:
            ep = l.iloc[i]
            af = min(af + a, m)
    return p

df['psar'] = ps(df['High'], df['Low'])


In [ ]:

df['sig'] = 0
df.loc[(df['roc'] > 0) & (df['Close'] > df['psar']), 'sig'] = 1
df.loc[(df['roc'] < 0) & (df['Close'] < df['psar']), 'sig'] = -1


In [ ]:

plt.figure()
plt.plot(df['Close'])
plt.scatter(df.index, df['psar'], s=5)
plt.show()


In [ ]:

plt.figure()
plt.plot(df['nvi'])
plt.show()
